In [ ]:
import cv2
import os
import numpy as np
from skimage.feature import hog
from skimage import exposure

# Define HOG parameters
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

# Define function to extract HOG features
def extract_features(image):
    """
    Extracts HOG features from the given image
    """
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    hog_features = hog(gray_image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, transform_sqrt=True, block_norm="L1")
    hog_features = np.array(hog_features)
    return hog_features

In [ ]:
# Load the dataset
X = []  # Input images
y = []  # Output labels
# Path to the directories containing the benign and malignant images

benign_path = "benign"
malignant_path = "malignant"

# Load the benign images and assign labels
for filename in os.listdir(benign_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(benign_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        X.append(image)
        y.append(0)

# Load the malignant images and assign labels
for filename in os.listdir(malignant_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(malignant_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        X.append(image)
        y.append(1)





# Split the dataset into training, validation, and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X_train[0])

224

In [ ]:
from sklearn.svm import SVC

# Define SVM classifier
svm = SVC(kernel='linear', probability=True)

# Extract features from the training set
X_train_features = []
for image in X_train:
    features = extract_features(image)
    features.shape
    X_train_features.append(features)
X_train_features = np.array(X_train_features)

# Train the SVM classifier
# svm.fit(X_train_features, y_train)

<ipython-input-12-90257eb66154>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_features = np.array(X_train_features)


In [ ]:
# Extract features from the test set
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
X_test_features = []
for image in X_test:
    features = extract_features(image)
    X_test_features.append(features)
X_test_features = np.array(X_test_features)

# Evaluate the SVM classifier on the test set
accuracy = svm.score(X_test_features, y_test)
y_pred = svm.predict(X_test_features)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display the evaluation metrics
print('Test Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Test Accuracy: 0.7348484848484849
Precision: 0.6888888888888889
Recall: 0.5961538461538461
F1 Score: 0.6391752577319588


In [ ]:
# Define function to predict the label and probability of a single image
def predict_single_image(image):
    # Extract HOG features from the image
    features = extract_features(image)
    # Reshape the features to match the shape of the training data
    features = features.reshape(1, -1)
    # Predict the label and probability using the trained SVM classifier
    label = svm.predict(features)[0]
    probability = svm.predict_proba(features)[0][label]
    return label, probability

# Load the single test image
image_path = 'test/b1.jpg'
image = cv2.imread(image_path)

# Predict the label and probability of the test image
if image is not None:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    label, probability = predict_single_image(image)
    print('Label:', label)
    print('Probability:', probability)

Label: 0
Probability: 0.8684873837460515
